# CS5228 Rental Prediction

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import f1_score, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from matplotlib.colors import ListedColormap
# load data
train_data = pd.read_csv('./data/processed/train.csv')
test_data = pd.read_csv('./data/processed/test.csv')

X_train, y_train = train_data.drop(columns='monthly_rent'), train_data['monthly_rent'].copy()
X_test = test_data

# Standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [6]:
name_SVM = 'SVM'
params = {
    "C": 2000,
    "kernel": 'rbf',
    "gamma": 'auto'
}

model_SVM = SVR(** params)
print(f'Training {name_SVM}...')

model_SVM.fit(X_train, y_train)
y_pred = model_SVM.predict(X_test)

y_train_pred = model_SVM.predict(X_train)

print(f'{name_SVM} training completed.')
result_df = pd.DataFrame({
        'Id': range(len(y_pred)),
        'Predicted': y_pred
    })
save_path = './data/predictions/' + name_SVM + '.csv'
result_df.to_csv(save_path, index=False)
print(f'{name_SVM} training result saved.\n')

print('{} evaluation:\n'.format(name_SVM))

mse_GradientBoosting = mean_squared_error(y_train, y_train_pred)
mae_GradientBoosting = mean_absolute_error(y_train, y_train_pred)
print(mse_GradientBoosting,mae_GradientBoosting)

# Calculate the RMSE based on another prediction
# pred_best = pd.read_csv('./data/predictions/Gradient Boosting.csv')
# y_pred_best = pred_best['Predicted']
# mse_GradientBoosting = mean_squared_error(y_pred, y_pred_best)
# rmse_GradientBoosting = np.sqrt(mse_GradientBoosting)
# print(rmse_GradientBoosting)

Training SVM...
SVM training completed.
SVM training result saved.

SVM evaluation:

222197.15115249116 343.9357513550652
